# 🧩 Ideal AI - Universal LLM Connector Demo

> **One Connector to Rule Them All**

This notebook demonstrates the **ideal-ai** package - a professional, pip-installable universal LLM connector.

## What You'll Learn

1. ✅ Modern package installation
2. ✅ Text generation across multiple providers
3. ✅ Vision/multimodal capabilities
4. ✅ Image generation
5. ✅ Audio transcription
6. ✅ Speech synthesis (TTS)
7. ✅ Video generation (async)
8. ✅ Runtime model injection
9. ✅ Smolagents integration

## 📦 Installation

Install the package (skip if already installed):

In [ ]:
# Install from PyPI (when published)
%pip install ideal-ai

# Or install from local source (development mode)
#%pip install -e ..

## 🔑 Setup COLAB API Keys

Load your API keys from environment or .env file:

In [ ]:
import os
from getpass import getpass

print("🚀 Ideal-AI Demo Configuration")
print("ℹ️ The script will attempt to load keys from Colab Secrets.")
print("ℹ️ If a permission popup appears and you click 'No' or 'Cancel', the key will be skipped automatically.\n")

# 1. SETUP: Helper function to load silently
def load_key_silently(env_var_name, secret_name, display_name):
    """Try to load from secrets. If fails, just skip. No blocking input."""
    try:
        from google.colab import userdata
        val = userdata.get(secret_name)
        if val:
            os.environ[env_var_name] = val
            print(f"✅ {display_name}: Loaded via Secrets.")
            return True
    except Exception:
        # This catches "SecretNotFoundError" AND "NotebookAccessError" (User said No)
        # Crucial: We do NOT call getpass() here. We just pass.
        pass
    
    print(f"⚠️ {display_name}: Skipped (Not found in Secrets or denied).")
    return False

# --- 2. EXECUTION: Loop through keys non-stop ---

# A. CORE KEYS
print("🔹 STEP 1: The Essentials")
load_key_silently("GOOGLE_API_KEY", "GOOGLE_API_KEY", "Google (Gemini)")
load_key_silently("OPENAI_API_KEY", "OPENAI_API_KEY", "OpenAI")
load_key_silently("ALIBABA_API_KEY", "ALIBABA_API_KEY", "Alibaba (Wan 2.1)")

# B. OPTIONAL KEYS
print("\n🔹 STEP 2: Bonus Keys")
OPTIONAL_KEYS = [
    ("INFOMANIAK_AI_TOKEN", "Infomaniak Token"),
    ("INFOMANIAK_PRODUCT_ID", "Infomaniak Product ID"),
    ("ANTHROPIC_API_KEY", "Anthropic"),
    ("MOONSHOT_API_KEY", "Moonshot"),
    ("PERPLEXITY_API_KEY", "Perplexity"),
    ("HF_TOKEN", "Hugging Face"),
    ("MINIMAX_API_KEY", "MiniMax")
]

count = 0
for env_key, label in OPTIONAL_KEYS:
    # Use the same secret name as env var for simplicity
    if load_key_silently(env_key, env_key, label):
        count += 1

print(f"\n✅ Configuration complete. {count} bonus keys loaded.")

# --- 3. OLLAMA (Local) ---
os.environ["OLLAMA_URL"] = "http://localhost:11434"

# --- 4. MANUAL FALLBACK (Optional) ---
# Only ask ONCE if the user really wants to type manually
print("\n💡 Tip: If you don't use Colab Secrets, you can set keys manually below.")
# Uncomment the lines below if you want to force manual input:
# if not os.getenv("OPENAI_API_KEY") and not os.getenv("GOOGLE_API_KEY"):
#     k = getpass("👉 Enter OpenAI or Google Key manually (or press Enter to skip): ")
#     if k.strip(): os.environ["OPENAI_API_KEY"] = k.strip()

## 🚀 Initialize Connector

**NEW**: No more `sys.path.append` hacks! Just import the package:

In [ ]:
# Modern import - works after pip install
from ideal_ai import IdealUniversalLLMConnector
import os

# --- FIX: Reconstruct dictionary from Environment Variables ---
API_KEYS = {
    "openai": os.getenv("OPENAI_API_KEY"),
    "google": os.getenv("GOOGLE_API_KEY"),
    "anthropic": os.getenv("ANTHROPIC_API_KEY"),
    "alibaba": os.getenv("ALIBABA_API_KEY"),
    "infomaniak": os.getenv("INFOMANIAK_AI_TOKEN"),
    "infomaniak_product": os.getenv("INFOMANIAK_PRODUCT_ID"),
    "moonshot": os.getenv("MOONSHOT_API_KEY"),
    "perplexity": os.getenv("PERPLEXITY_API_KEY"),
    "hugging_face": os.getenv("HF_TOKEN"),
    "minimax": os.getenv("MINIMAX_API_KEY"),
}

# Initialize connector
connector = IdealUniversalLLMConnector(
    api_keys=API_KEYS,
    ollama_url=os.getenv("OLLAMA_URL", "http://localhost:11434")
)

print("✅ Connector initialized successfully!")
print(f"🤖 Ready to use with {len(connector.model_configs)} pre-configured models")

## 📋 List Available Models

See what's available out of the box:

In [ ]:
import json

available = connector.list_available_families(include_models=True)

print("\n=== Available Models by Modality ===")
for modality, families in available.items():
    print(f"\n🎯 {modality.upper()}:")
    for family_name, details in families.items():
        models = details.get("models", [])
        if isinstance(models, list) and models:
            # Extract model names from list
            model_names = [m.get("model") if isinstance(m, dict) else m for m in models[:3]]
            print(f"  • {family_name}: {', '.join(model_names)}...")

## 1️⃣ Text Generation: The "Universal Loop" ♾️

This is the core promise of **ideal-ai**: One unified interface for all providers.

Instead of writing specific code for OpenAI, Google, or Alibaba, we iterate through them
using **the exact same code structure**.

*Note: The loop below automatically detects which API keys you provided in the setup step and only runs on available providers.*

In [ ]:
import os

# --- THE SMART LOOP CONFIGURATION ---
# We define potential targets. The code checks environment variables loaded
# in the "Setup" step. It ONLY runs providers you actually configured.
# NO extra API keys will be requested here.
potential_targets = [
    ("openai", "gpt-4o", ["OPENAI_API_KEY"]),
    ("google", "gemini-2.5-flash", ["GOOGLE_API_KEY"]),
    ("anthropic", "claude-haiku-4-5-20251001", ["ANTHROPIC_API_KEY"]),
    ("alibaba", "qwen-plus", ["ALIBABA_API_KEY"]),
    ("infomaniak", "mixtral", ["INFOMANIAK_AI_TOKEN", "INFOMANIAK_PRODUCT_ID"]),
    ("mistral", "mistral-large-latest", ["MISTRAL_API_KEY"])
]

# Build the active list dynamically based on loaded keys
active_targets = []
for provider, model, required_keys in potential_targets:
    if all(os.getenv(k) for k in required_keys):
        active_targets.append({"provider": provider, "model": model})

# 💡 MARKETING PROMPT: We ask the AI to explain why THIS tool is useful!
prompt = "Explain the benefit of having one Universal API for all AI providers in one funny sentence."

print(f"🎤 Prompt: \"{prompt}\"\n")

if not active_targets:
    print("⚠️ No API keys detected! Please run the 'Setup Keys' cell above.")
else:
    print(f"✅ Detected {len(active_targets)} active providers. Running the Universal Loop...\n")

    # --- THE MAGIC: ONE CODE BLOCK FOR EVERYONE ---
    for target in active_targets:
        print(f"⏳ Calling {target['provider'].upper()} ({target['model']})...")
        try:
            response = connector.invoke(
                provider=target["provider"],
                model_id=target["model"],
                messages=[{"role": "user", "content": prompt}]
            )
            print(f"   🤖 Result: {response['text']}\n")
        except Exception as e:
            print(f"   ❌ Error: {e}\n")

In [ ]:
# Compare with cloud providers
providers_to_test = [
    ("openai", "gpt-4o"),
    ("google", "gemini-2.5-flash"),
    ("anthropic", "claude-haiku-4-5-20251001"),
]

prompt = "What is the meaning of life? Answer in max 50 characters."

for provider, model in providers_to_test:
    try:
        result = connector.invoke(
            provider=provider,
            model_id=model,
            messages=[{"role": "user", "content": prompt}]
        )
        print(f"\n✅ {provider}/{model}:")
        print(f"   {result['text']}")
    except Exception as e:
        print(f"\n❌ {provider}/{model}: {e}")

## 2️⃣ Vision/Multimodal

Analyze images with vision models:

In [ ]:
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display

# Download test image from ia-agence.ai
image_url = "https://ia-agence.ai/wp-content/uploads/2025/12/demo_image_ideal_ai_connector.jpg"
response = requests.get(image_url)
image_bytes = response.content

# Display image
display(Image.open(BytesIO(image_bytes)))

# Analyze with vision model
analysis = connector.invoke_image(
    provider="google",  # or "openai", "anthropic", "ollama"
    model_id="gemini-2.5-flash",
    image_input=image_bytes,
    prompt="Describe what you see in this image in English."
)

print("\n👁️ Vision Analysis:")
print(analysis["text"])

## 3️⃣ Image Generation

Generate images from text prompts:

In [ ]:
import base64
from IPython.display import Image as DisplayImage

# Generate image
result = connector.invoke_image_generation(
    provider="openai",  # or "infomaniak"
    model_id="dall-e-3",
    prompt="A cute robot sitting on a rock, looking at a castle, warm sunset light, Disney style",
    width=1024,
    height=1024
)

if result.get("images"):
    # Decode and display
    b64_string = result["images"][0]
    img_bytes = base64.b64decode(b64_string)
    
    print("🎨 Generated Image:")
    display(DisplayImage(data=img_bytes))
    
    # Save to file
    output_path = "generated_image.png"
    Image.open(BytesIO(img_bytes)).save(output_path)
    print(f"✅ Saved to: {output_path}")
else:
    print("❌ No image generated")

## 4️⃣ Audio Transcription

Convert speech to text:

In [ ]:
import requests
from pathlib import Path

# Define file paths
local_file = "demo_audio.m4a"
remote_url = "https://ia-agence.ai/wp-content/uploads/2025/12/demo_sound_ideal_ai_connector.m4a"

# Download demo file automatically
if not Path(local_file).exists():
    with open(local_file, "wb") as f:
        f.write(requests.get(remote_url).content)

# Run transcription
transcription = connector.invoke_audio(
    provider="infomaniak",
    model_id="whisper",
    audio_file_path=local_file,
    language="en"
)

print("🎤 Transcription:")
print(transcription["text"])

## 5️⃣ Speech Synthesis (TTS)

Convert text to speech:

In [ ]:
from IPython.display import Audio

# Generate speech
audio_result = connector.invoke_speech_generation(
    provider="openai",
    model_id="tts-1",
    text="Hello! This is a test of the text-to-speech system using the Ideal AI connector.",
    voice="nova"  # Available: alloy, echo, fable, onyx, nova, shimmer
)

if audio_result.get("audio_bytes"):
    print("🗣️ Generated Speech:")
    display(Audio(data=audio_result["audio_bytes"]))
    
    # Save to file
    with open("generated_speech.mp3", "wb") as f:
        f.write(audio_result["audio_bytes"])
    print("✅ Saved to: generated_speech.mp3")
else:
    print("❌ No audio generated")

## 6️⃣ Video Generation

Generate video from text (async task with automatic polling):

In [ ]:
from IPython.display import Video

print("🎬 Starting video generation...")
print("⏳ This may take 1-5 minutes (polling handled automatically)")

video_result = connector.invoke_video_generation(
    provider="alibaba",
    model_id="wan2.1-t2v-turbo",  # or "wan2.2-t2v-plus", "wan2.5-t2v-preview"
    prompt="A futuristic robot standing in a high-tech urban environment at night",
    size="832*480"  # or "1280*720"
)

if video_result.get("videos"):
    video_url = video_result["videos"][0]
    print(f"\n🎉 Video generated!")
    print(f"📹 URL: {video_url}")
    
    # Download and display
    video_bytes = requests.get(video_url, timeout=60).content
    with open("generated_video.mp4", "wb") as f:
        f.write(video_bytes)
    
    print("✅ Saved to: generated_video.mp4")
    display(Video("generated_video.mp4", embed=True, width=400))
else:
    print("❌ No video generated")

## 7️⃣ Runtime Model Injection

**NEW FEATURE**: Add custom models without modifying source code!

This demonstrates the maintenance system - you can now add models just by editing JSON or injecting at runtime:

In [ ]:
# Example: Add a custom Ollama model at runtime
connector.register_model(
    "ollama:custom-model",
    {
        "families": {
            "text": "ollama_text"  # Reuse existing recipe
        }
    }
)

print("✅ Custom model registered!")

# Example: Add a cloud provider model
connector.register_model(
    "huggingface:custom-gpt",
    {
        "api_key_name": "hugging_face",
        "families": {
            "text": "openai_compatible"
        },
        "url_template": "https://router.huggingface.co/v1/chat/completions",
        "api_model_name": "openai/gpt-oss-20b"
    }
)

print("✅ Cloud provider model registered!")
print("\n💡 These models can now be used immediately with connector.invoke()")

## 8️⃣ Multi-Turn Conversation

Build conversational agents:

In [ ]:
# Initialize conversation history
conversation = []

def chat(user_message: str, provider: str = "openai", model: str = "gpt-4o"):
    """Send a message and get AI response."""
    conversation.append({"role": "user", "content": user_message})
    
    response = connector.invoke(
        provider=provider,
        model_id=model,
        messages=conversation,
        temperature=0.7
    )
    
    ai_message = response["text"]
    conversation.append({"role": "assistant", "content": ai_message})
    
    print(f"🤖 AI: {ai_message}")
    return ai_message

# Example conversation
print("👤 User: Hello! What's your name?")
chat("Hello! What's your name?")

print("\n👤 User: Can you help me with Python?")
chat("Can you help me with Python?")

## 9️⃣ Smolagents Integration

Use with smolagents for building AI agents:

In [ ]:
from ideal_ai import IdealSmolagentsWrapper
from smolagents import CodeAgent, tool

# 1. Define a custom tool (Robust & Dependency-free!)
@tool
def get_weather(location: str) -> str:
    """
    Get the current weather for a specific location.
    Args:
        location: The name of the city (e.g. 'Paris', 'New York')
    """
    # This is a mock for the demo - zero failure risk
    return f"The weather in {location} is sunny with a perfect temperature for coding!"

# 2. Wrap connector for smolagents
model = IdealSmolagentsWrapper(
    connector=connector,
    provider="openai", # Ensure you have a key set for this provider
    model_id="gpt-4o"
)

# 3. Create agent with our custom tool
agent = CodeAgent(
    tools=[get_weather],
    model=model
)

# 4. Run agent task
print("🤖 Agent working...")
result = agent.run("What is the weather in Paris? And tell me a joke about it.")
print(result)

## 🔧 Advanced: Custom Parser

Handle non-standard API response formats:

In [ ]:
# Define custom parser for a specific provider
def my_custom_parser(raw_response):
    """Extract text from custom API response format."""
    # Example: handle nested response
    return raw_response.get("data", {}).get("output", {}).get("text", "")

# Create connector with custom parser
custom_connector = IdealUniversalLLMConnector(
    api_keys=API_KEYS,
    parsers={
        "provider:model-id": my_custom_parser  # Specific model
        # or "provider": my_custom_parser  # All models from provider
    }
)

print("✅ Connector created with custom parser")
print("💡 The custom parser will be used automatically when calling that model")

## 🐛 Debugging

Enable debug mode to inspect API calls:

In [ ]:
# Enable debug to see raw payloads and responses
try:
    debug_response = connector.invoke(
        provider="google",
        model_id="gemini-2.5-flash",
        messages=[{"role": "user", "content": "Hi! System check."}],
        debug=True  # ← Enable debug mode to see the JSON payloads
    )

    print("\n📊 Debug output shown above")
    print(f"📝 Response: {debug_response['text']}")
except Exception as e:
    print(f"❌ Debug test failed: {e}")
    print("Tip: Check if your GOOGLE_API_KEY is set.")

## 🧪 Interactive Testing Interface

**Bonus**: All-in-one graphical testing tool for quick experimentation

This section provides a comprehensive widget-based interface to test all modalities without writing code. Perfect for quick prototyping and model comparison.

**Features:**
- ✍️ Text generation (one-shot + multi-turn chat)
- 🎯 Multi-model benchmark comparison
- 👁️ Vision/image analysis
- 🎨 Image generation
- 🎬 Video generation
- 🗣️ Speech synthesis (TTS)
- 🎤 Audio transcription (STT)
- 🗣️🎙️ Voice chat (full audio-to-audio pipeline)

In [ ]:
# Import required widget libraries
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout 
from IPython.display import display, Markdown, clear_output, Image as DisplayImage, Audio, Video, HTML
import sys
import base64
import requests 
import io  
import os 
import json
from pathlib import Path
from tabulate import tabulate 
from datetime import datetime

# Audio-specific imports for voice chat
try:
    import sounddevice as sd
    import numpy as np
    from scipy.io.wavfile import write
    import pygame
    import time
    AUDIO_LIBS_AVAILABLE = True
except ImportError:
    AUDIO_LIBS_AVAILABLE = False
    print("⚠️ Audio libraries not available. Voice chat will be disabled.")

# PIL for image handling
try:
    from PIL import Image as PILImage
except ImportError:
    PILImage = None

# Verify connector is initialized
if 'connector' not in locals() or not connector:
    print("❌ ERROR: Connector not initialized. Please run the initialization cell first.")
else:
    print("✅ Building unified testing interface...")
    
    # Initialize pygame mixer for audio playback
    if AUDIO_LIBS_AVAILABLE:
        try:
            pygame.mixer.init()
            print("🎧 Pygame mixer initialized for audio playback.")
        except Exception as e:
            print(f"⚠️ Warning: Failed to initialize pygame.mixer: {e}")

    # Setup output directories for generated content
    OUTPUT_DIRS = {}
    try:
        home_dir = Path.home()
        base_output = home_dir / "JupyterLab" / "notebooks" / "data"
        OUTPUT_DIRS["image"] = base_output / "images_gen_output"
        OUTPUT_DIRS["video"] = base_output / "videos_gen_output"
        OUTPUT_DIRS["speech"] = base_output / "audios_gen_output"
        OUTPUT_DIRS["audio_in"] = base_output / "audios_input"
        for path in OUTPUT_DIRS.values():
            os.makedirs(path, exist_ok=True)
        print(f"📂 Output directories configured in {base_output}")
    except Exception as e:
        print(f"❌ Error creating output directories: {e}")

    # Retrieve models by modality from connector
    MODELS_BY_MODALITY = {
        "text": [], "vision": [], "image_gen": [], 
        "video_gen": [], "speech_gen": [], "audio": [] 
    }
    try:
        available_models = connector.list_available_families(include_models=True)
        for modality, families in available_models.items():
            if modality in MODELS_BY_MODALITY: 
                for family_name, details in families.items():
                    for model_item in details.get("models", []):
                        model_pair = model_item.get("model") if isinstance(model_item, dict) else model_item
                        if model_pair:
                            MODELS_BY_MODALITY[modality].append(model_pair)
        for mod in MODELS_BY_MODALITY:
            MODELS_BY_MODALITY[mod] = sorted(list(set(MODELS_BY_MODALITY[mod])))
        print(f"✅ Models loaded: {sum(len(v) for v in MODELS_BY_MODALITY.values())} total")
    except Exception as e:
        print(f"❌ Error retrieving model lists: {e}")
        # Fallback lists
        MODELS_BY_MODALITY["text"] = ['ollama:llama3.2', 'openai:gpt-4o', 'google:gemini-2.5-flash']
        MODELS_BY_MODALITY["vision"] = ['ollama:llava', 'google:gemini-2.5-flash', 'openai:gpt-4o']
        MODELS_BY_MODALITY["image_gen"] = ['infomaniak:sdxl-lightning', 'openai:dall-e-3']
        MODELS_BY_MODALITY["video_gen"] = ['alibaba:wan2.1-t2v-turbo']
        MODELS_BY_MODALITY["speech_gen"] = ['openai:tts-1']
        MODELS_BY_MODALITY["audio"] = ['infomaniak:whisper'] 

    # Modality icons for accordion
    MODALITY_ICONS = {
        "text": "✍️", "vision": "👁️", "audio": "🔊", 
        "image_gen": "🎨", "video_gen": "🎬", "speech_gen": "🗣️", "other": "⚙️"
    }

    # =================================================================
    # TAB 1: TEXT TESTER (One-Shot) + ACCORDION
    # =================================================================
    
    text_llm_selector = widgets.Dropdown(
        options=MODELS_BY_MODALITY["text"], 
        value=MODELS_BY_MODALITY["text"][0], 
        description='Model:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    text_prompt_input = widgets.Textarea(
        value="Write a short poem about the beauty of code.", 
        description='Prompt:', 
        style={'description_width': 'auto'}, 
        layout=Layout(width='90%', height='100px')
    )
    text_invoke_button = widgets.Button(
        description='🚀 Invoke', 
        button_style='success', 
        layout=Layout(margin='10px 0 0 55px')
    )
    text_output_area = widgets.Output(layout=Layout(margin='10px'))
    
    def on_invoke_text_clicked(b):
        with text_output_area:
            clear_output(wait=True)
            selected_pair = text_llm_selector.value
            try: 
                provider, model_id = selected_pair.split(":", 1)
            except ValueError: 
                print(f"❌ Format error: {selected_pair}")
                return
            print(f"--- Testing {model_id} via {provider} ---")
            messages = [{"role": "user", "content": text_prompt_input.value}]
            try:
                response = connector.invoke(
                    provider=provider, 
                    model_id=model_id, 
                    messages=messages, 
                    temperature=1.0, 
                    debug=False
                )
                result_text = response.get("text", "No text returned.")
                display(Markdown(f"### ✅ Response from **{model_id}**\n---\n{result_text}"))
            except Exception as e:
                display(Markdown(f"### ❌ Error from {model_id} ({provider})"))
                print(f"Detail: {e}")
    
    text_invoke_button.on_click(on_invoke_text_clicked)
    
    # Build accordion showing available families and models
    try:
        families_for_accordion = connector.list_available_families(include_models=True)
        accordion_children = []
        for modality, families in families_for_accordion.items():
            html_content = ""
            if not families: 
                html_content = "<i>No families defined.</i>"
            else:
                for family_name, details in families.items():
                    html_content += f"<h4 style='margin-bottom: 5px;'>► Family: <code>{family_name}</code></h4>"
                    url_template = details.get("url_template", "[N/A]")
                    html_content += "<ul>"
                    html_content += f"<li><strong>URL Template</strong>: <code>{url_template}</code></li>"
                    models = details.get("models", [])
                    if not models: 
                        html_content += "<li><strong>Models</strong>: (None)</li>"
                    else:
                        html_content += "<li><strong>Models</strong>:</li><ul style='margin-top: 5px;'>"
                        for model_item in models:
                            if isinstance(model_item, dict):
                                model_name = model_item.get("model", "Unknown")
                                model_url = model_item.get("url_template", "")
                                if "See URLs" in url_template: 
                                    html_content += f"<li><code>{model_name}</code> (URL: <code>{model_url}</code>)</li>"
                                else: 
                                    html_content += f"<li><code>{model_name}</code></li>"
                            else: 
                                html_content += f"<li><code>{model_item}</code></li>"
                        html_content += "</ul>"
                    html_content += "</ul><hr style='border: none; border-top: 1px dashed #ccc; margin: 10px 0;'>"
            accordion_children.append(widgets.HTML(value=html_content))
        accordion_widget = widgets.Accordion(children=accordion_children, selected_index=None)
        for i, modality in enumerate(families_for_accordion.keys()):
            icon = MODALITY_ICONS.get(modality, "⚙️")
            accordion_widget.set_title(i, f"{icon} Modality: {modality.upper()}")
        accordion_title = widgets.HTML("<h3>🧩 Available Families and Models</h3>")
    except Exception as e:
        accordion_widget = widgets.HTML(f"<b>Accordion Error:</b> {e}")
        accordion_title = widgets.HTML("")
    
    tab_text = widgets.VBox([
        text_llm_selector, text_prompt_input, text_invoke_button, 
        text_output_area, widgets.HTML("<hr>"), accordion_title, accordion_widget
    ])

    # =================================================================
    # TAB 2: CHATBOT (Multi-Turn Conversation)
    # =================================================================
    
    chat_history = []
    
    chat_llm_selector = widgets.Dropdown(
        options=MODELS_BY_MODALITY["text"], 
        value=MODELS_BY_MODALITY["text"][0], 
        description='Model:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='100%')
    )
    chat_output_area = widgets.Output(
        layout=Layout(width='100%', height='400px', overflow_y='auto', 
                     border='1px solid #ccc', padding='10px')
    )
    chat_output_area.add_class("chat-output") 
    chat_prompt_input = widgets.Text(
        value='', 
        description='You:', 
        placeholder='Enter your message and press [Enter]...', 
        style={'description_width': 'auto'}, 
        layout=Layout(width='100%'),
        continuous_update=False
    )
    chat_send_button = widgets.Button(description='Send', button_style='primary')
    chat_clear_button = widgets.Button(description='Clear', button_style='danger')

    def send_chat_message(source_widget):
        provider, model_id = chat_llm_selector.value.split(":", 1)
        prompt = chat_prompt_input.value
        if not prompt:
            return
        chat_history.append({"role": "user", "content": prompt})
        with chat_output_area:
            display(HTML(f"<p style='margin: 0;'><b>You:</b> {prompt}</p>"))
        chat_prompt_input.value = ''
        messages_to_send = chat_history.copy()
        if len(messages_to_send) == 1: 
            messages_to_send.insert(0, {
                "role": "system", 
                "content": "You are a helpful assistant. Keep responses concise (max 150 characters)."
            })
        with chat_output_area:
            print(f"--- {model_id} thinking... ---")
            try:
                response = connector.invoke(
                    provider=provider, model_id=model_id, 
                    messages=messages_to_send, temperature=0.7, debug=False
                )
                result_text = response.get("text", "Sorry, I couldn't respond.")
                chat_history.append({"role": "assistant", "content": result_text})
                clear_output(wait=True) 
                for msg in chat_history:
                    if msg['role'] == 'user':
                        display(HTML(f"<p style='margin: 0;'><b>You:</b> {msg['content']}</p>"))
                    else:
                        display(HTML(
                            f"<p style='margin: 10px 0;'><b>{model_id}:</b> {msg['content']}</p>"
                            f"<hr style='border: none; border-top: 1px dashed #ccc; margin: 10px 0;'>"
                        ))
            except Exception as e:
                display(Markdown(f"### ❌ Error from {model_id} ({provider})"))
                print(f"Detail: {e}")

    def on_chat_clear_clicked(b):
        chat_history.clear()
        with chat_output_area:
            clear_output()
            print("Conversation history cleared.")

    chat_send_button.on_click(lambda b: send_chat_message(b))
    chat_clear_button.on_click(on_chat_clear_clicked)
    chat_prompt_input.observe(lambda change: send_chat_message(change), names='value')
    
    chat_input_box = widgets.HBox(
        [chat_prompt_input, chat_send_button, chat_clear_button], 
        layout=Layout(width='100%', padding='5px 0 0 0')
    )
    tab_chat = AppLayout(
        header=chat_llm_selector, center=chat_output_area, footer=chat_input_box,
        pane_heights=['50px', '400px', '50px'] 
    )

    # =================================================================
    # TAB 3: BENCHMARK (Multi-Model Comparison)
    # =================================================================
    
    benchmark_model_selector = widgets.SelectMultiple(
        options=MODELS_BY_MODALITY["text"], 
        value=MODELS_BY_MODALITY["text"][:3], 
        description='Models to Test:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='90%', height='200px')
    )
    benchmark_prompt_input = widgets.Textarea(
        value="What is the meaning of life? Answer in max 100 characters.", 
        description='Prompt:', style={'description_width': 'auto'}, 
        layout=Layout(width='90%', height='100px')
    )
    benchmark_invoke_button = widgets.Button(
        description='🚀 Run Benchmark', button_style='danger', 
        layout=Layout(margin='10px 0 0 55px')
    )
    benchmark_output_area = widgets.Output(layout=Layout(margin='10px'))
    
    def on_invoke_benchmark_clicked(b):
        with benchmark_output_area:
            clear_output(wait=True)
            models_to_test = benchmark_model_selector.value
            if not models_to_test: 
                print("❌ Please select at least one model.")
                return
            question = benchmark_prompt_input.value
            messages = [{"role": "user", "content": question}]
            results = []
            print(f"📊 Running benchmark for {len(models_to_test)} models...")
            for pair in models_to_test:
                try: 
                    provider, model_id = pair.split(":", 1)
                except Exception: 
                    continue
                print(f"⏳ Testing {provider}/{model_id}...")
                try:
                    out = connector.invoke(provider, model_id, messages, 
                                         temperature=1.0, request_timeout=180) 
                    text = out.get("text", "")[:200].replace("\n", " ")
                    results.append([provider, model_id, "✅", text])
                except Exception as e:
                    results.append([provider, model_id, "❌", str(e)])
            headers = ["Provider", "Model", "Status", "Response (truncated)"]
            print("\n📊 Benchmark Results:\n")
            display(Markdown(tabulate(results, headers=headers, tablefmt="github"))) 
    
    benchmark_invoke_button.on_click(on_invoke_benchmark_clicked)
    tab_benchmark = widgets.VBox([
        benchmark_model_selector, benchmark_prompt_input, 
        benchmark_invoke_button, benchmark_output_area
    ])

    # =================================================================
    # TAB 4: VISION TESTER
    # =================================================================

    vision_llm_selector = widgets.Dropdown(
        options=MODELS_BY_MODALITY["vision"], 
        value=MODELS_BY_MODALITY["vision"][0], 
        description='Vision Model:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    vision_url_input = widgets.Text(
        value='https://ia-agence.ai/wp-content/uploads/2025/12/demo_image_ideal_ai_connector.jpg', 
        description='Image URL:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='90%')
    )
    vision_prompt_input = widgets.Textarea(
        value='Describe what you see in this image in detail.', 
        description='Prompt:', style={'description_width': 'auto'}, 
        layout=Layout(width='90%', height='80px')
    )
    vision_invoke_button = widgets.Button(
        description="🚀 Analyze Image", button_style='info', 
        layout=Layout(margin='10px 0 0 55px')
    )
    vision_image_area = widgets.Output(layout=Layout(width='400px', margin='10px'))
    vision_text_area = widgets.Output(layout=Layout(margin='10px'))
    
    def on_invoke_vision_clicked(b):
        with vision_image_area: 
            clear_output(wait=True)
        with vision_text_area: 
            clear_output(wait=True)
        selected_pair = vision_llm_selector.value
        url = vision_url_input.value
        prompt = vision_prompt_input.value
        try: 
            provider, model_id = selected_pair.split(":", 1)
        except ValueError:
            with vision_text_area: 
                print(f"❌ Format error: {selected_pair}")
            return
        try:
            with vision_text_area: 
                print(f"Downloading from {url}...")
            response = requests.get(url, timeout=20)
            response.raise_for_status()
            image_data_bytes = response.content
            with vision_text_area: 
                print("Image downloaded.")
            with vision_image_area: 
                display(DisplayImage(data=image_data_bytes, width=400))
            image_pil_object = PILImage.open(io.BytesIO(image_data_bytes)) if PILImage else None
            input_for_connector = image_data_bytes
            if image_pil_object and provider == "google":
                input_for_connector = image_pil_object
            with vision_text_area: 
                print(f"Calling {provider}:{model_id}...")
            out = connector.invoke_image(
                provider=provider, model_id=model_id, 
                image_input=input_for_connector, prompt=prompt, debug=False
            )
            with vision_text_area: 
                clear_output(wait=True)
                display(Markdown(f"### ✅ Response from **{model_id}**\n---\n{out.get('text')}"))
        except Exception as e:
            with vision_text_area: 
                clear_output(wait=True)
                display(Markdown(f"### ❌ Vision Error"))
                print(f"Detail: {e}")
    
    vision_invoke_button.on_click(on_invoke_vision_clicked)
    tab_vision = widgets.VBox([
        vision_llm_selector, vision_url_input, vision_prompt_input, 
        vision_invoke_button, widgets.HBox([vision_image_area, vision_text_area])
    ])

    # =================================================================
    # TAB 5: IMAGE GENERATION
    # =================================================================
    
    image_gen_llm_selector = widgets.Dropdown(
        options=MODELS_BY_MODALITY["image_gen"], 
        value=MODELS_BY_MODALITY["image_gen"][0], 
        description='Image Model:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    image_gen_prompt_input = widgets.Textarea(
        value='A Disney-style robot sitting on a rock, looking at a distant castle, warm sunset light.', 
        description='Prompt:', style={'description_width': 'auto'}, 
        layout=Layout(width='90%', height='100px')
    )
    image_gen_invoke_button = widgets.Button(
        description="🚀 Generate Image", button_style='warning', 
        layout=Layout(margin='10px 0 0 55px')
    )
    image_gen_display_area = widgets.Output(layout=Layout(margin='10px'))
    
    def on_invoke_image_gen_clicked(b):
        with image_gen_display_area:
            clear_output(wait=True)
            selected_pair = image_gen_llm_selector.value
            prompt = image_gen_prompt_input.value
            try: 
                provider, model_id = selected_pair.split(":", 1)
            except ValueError: 
                print(f"❌ Format error: {selected_pair}")
                return
            print(f"🎨 Starting generation via {provider} with {model_id}...")
            try:
                result = connector.invoke_image_generation(
                    provider=provider, model_id=model_id, prompt=prompt, debug=False
                )
                if result and result.get("images"):
                    b64_str = result["images"][0]
                    img_bytes = base64.b64decode(b64_str)
                    print("🖼️ Displaying image...")
                    display(DisplayImage(data=img_bytes))
                    timestamp = datetime.now().strftime("%Y_%m_%d_%H%M%S")
                    safe_model_id = model_id.replace(":", "_")
                    filename = f"{timestamp}_{provider}_{safe_model_id}.png"
                    final_output_path = OUTPUT_DIRS["image"] / filename
                    img_pil = PILImage.open(io.BytesIO(img_bytes))
                    img_pil.save(final_output_path)
                    print(f"✅ Image saved to: {final_output_path}")
                else: 
                    print(f"❌ No image generated. Raw: {result.get('raw')}")
            except Exception as e: 
                display(Markdown(f"### ❌ Image Generation Error"))
                print(f"Detail: {e}")
    
    image_gen_invoke_button.on_click(on_invoke_image_gen_clicked)
    tab_image_gen = widgets.VBox([
        image_gen_llm_selector, image_gen_prompt_input, 
        image_gen_invoke_button, image_gen_display_area
    ])

    # =================================================================
    # TAB 6: VIDEO GENERATION
    # =================================================================
    
    video_gen_llm_selector = widgets.Dropdown(
        options=MODELS_BY_MODALITY["video_gen"], 
        value=MODELS_BY_MODALITY["video_gen"][0], 
        description='Video Model:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    video_gen_prompt_input = widgets.Textarea(
        value='A sleek futuristic robot standing in a high-tech urban environment at night.', 
        description='Prompt:', style={'description_width': 'auto'}, 
        layout=Layout(width='90%', height='100px')
    )
    video_gen_invoke_button = widgets.Button(
        description='🚀 Generate Video (slow)', button_style='danger', 
        layout=Layout(margin='10px 0 0 55px')
    )
    video_gen_display_area = widgets.Output(layout=Layout(margin='10px'))
    
    def on_invoke_video_gen_clicked(b):
        with video_gen_display_area:
            clear_output(wait=True)
            selected_pair = video_gen_llm_selector.value
            prompt = video_gen_prompt_input.value
            try: 
                provider, model_id = selected_pair.split(":", 1)
            except ValueError: 
                print(f"❌ Format error: {selected_pair}")
                return
            print(f"🎬 Starting video generation via {provider} with {model_id}...")
            print("⏳ This may take 1-5 minutes...")
            try:
                result = connector.invoke_video_generation(
                    provider=provider, model_id=model_id, prompt=prompt, debug=False
                )
                if result and result.get("videos"):
                    video_url = result["videos"][0]
                    print(f"🎉 Video generated! URL: {video_url}")
                    print("⬇️ Downloading...")
                    video_bytes = requests.get(video_url, timeout=60).content
                    timestamp = datetime.now().strftime("%Y_%m_%d_%H%M%S")
                    safe_model_id = model_id.replace(":", "_")
                    filename = f"{timestamp}_{provider}_{safe_model_id}.mp4"
                    final_output_path = OUTPUT_DIRS["video"] / filename
                    with open(final_output_path, "wb") as f: 
                        f.write(video_bytes)
                    print(f"✅ Video saved to: {final_output_path}")
                    print("\n🎬 Displaying video:")
                    display(Video(final_output_path, embed=True, width=400))
                else: 
                    print(f"❌ No video generated. Raw: {result.get('raw')}")
            except Exception as e: 
                display(Markdown(f"### ❌ Video Generation Error"))
                print(f"Detail: {e}")
    
    video_gen_invoke_button.on_click(on_invoke_video_gen_clicked)
    tab_video_gen = widgets.VBox([
        video_gen_llm_selector, video_gen_prompt_input, 
        video_gen_invoke_button, video_gen_display_area
    ])

    # =================================================================
    # TAB 7: SPEECH SYNTHESIS (TTS)
    # =================================================================
    
    speech_gen_llm_selector = widgets.Dropdown(
        options=MODELS_BY_MODALITY["speech_gen"], 
        value=MODELS_BY_MODALITY["speech_gen"][0], 
        description='TTS Model:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    speech_gen_voice_selector = widgets.Dropdown(
        options=['alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer'], 
        value='nova', description='Voice:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    speech_gen_prompt_input = widgets.Textarea(
        value="Hello! This is a test of speech synthesis from the unified testing interface.", 
        description='Text:', style={'description_width': 'auto'}, 
        layout=Layout(width='90%', height='100px')
    )
    speech_gen_invoke_button = widgets.Button(
        description="🚀 Generate Audio", button_style='primary', 
        layout=Layout(margin='10px 0 0 55px')
    )
    speech_gen_display_area = widgets.Output(layout=Layout(margin='10px'))
    
    def on_invoke_speech_gen_clicked(b):
        with speech_gen_display_area:
            clear_output(wait=True)
            selected_pair = speech_gen_llm_selector.value
            prompt = speech_gen_prompt_input.value
            voice = speech_gen_voice_selector.value
            try: 
                provider, model_id = selected_pair.split(":", 1)
            except ValueError: 
                print(f"❌ Format error: {selected_pair}")
                return
            print(f"🗣️ Starting speech synthesis via {provider} with {model_id} (Voice: {voice})...")
            try:
                result = connector.invoke_speech_generation(
                    provider=provider, model_id=model_id, 
                    text=prompt, voice=voice, debug=False
                )
                if result and result.get("audio_bytes"):
                    audio_bytes = result["audio_bytes"]
                    print("🎧 Displaying audio player...")
                    display(Audio(data=audio_bytes))
                    timestamp = datetime.now().strftime("%Y_%m_%d_%H%M%S")
                    safe_model_id = model_id.replace(":", "_")
                    filename = f"{timestamp}_{provider}_{safe_model_id}_{voice}.mp3"
                    final_output_path = OUTPUT_DIRS["speech"] / filename
                    with open(final_output_path, "wb") as f: 
                        f.write(audio_bytes)
                    print(f"✅ Audio saved to: {final_output_path}")
                else: 
                    print(f"❌ No audio generated. Raw: {result.get('raw')}")
            except Exception as e: 
                display(Markdown(f"### ❌ Speech Synthesis Error"))
                print(f"Detail: {e}")
    
    speech_gen_invoke_button.on_click(on_invoke_speech_gen_clicked)
    tab_speech_gen = widgets.VBox([
        speech_gen_llm_selector, speech_gen_voice_selector, 
        speech_gen_prompt_input, speech_gen_invoke_button, speech_gen_display_area
    ])

    # =================================================================
    # TAB 8: AUDIO TRANSCRIPTION (STT)
    # =================================================================
    
    # 1. Automatic Audio File Management (Download from ia-agence.ai)
    audio_filename = "demo_sound_ideal_ai_connector.m4a"
    audio_url = "https://ia-agence.ai/wp-content/uploads/2025/12/demo_sound_ideal_ai_connector.m4a"
    
    if not os.path.exists(audio_filename):
        print(f"⬇️ Downloading demo audio from ia-agence.ai...")
        try:
            r = requests.get(audio_url, timeout=10)
            r.raise_for_status() 
            with open(audio_filename, 'wb') as f:
                f.write(r.content)
            print("✅ Audio file downloaded successfully.")
        except Exception as e:
            print(f"⚠️ Failed to download audio demo: {e}")
            # Ensure the variable exists even if download fails
            if not os.path.exists(audio_filename):
                audio_filename = "demo_sound_ideal_ai_connector.m4a" 

    # 2. Widgets
    audio_llm_selector = widgets.Dropdown(
        options=MODELS_BY_MODALITY["audio"], 
        value=MODELS_BY_MODALITY["audio"][0] if MODELS_BY_MODALITY["audio"] else None, 
        description='STT Model:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    
    audio_file_input = widgets.Text(
        value=audio_filename,  # Points to the local downloaded file
        description='Audio File Path:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='90%')
    )
    
    audio_language_input = widgets.Text(
        value='en', description='Language (ISO code):', 
        placeholder='e.g., en, fr, es...', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    
    audio_invoke_button = widgets.Button(
        description="🚀 Transcribe Audio", button_style='info', 
        layout=Layout(margin='10px 0 0 55px')
    )
    
    audio_output_area = widgets.Output(layout=Layout(margin='10px'))

    # 3. La logique ne s'exécute que quand on CLIQUE
    def on_invoke_audio_clicked(b):
        with audio_output_area:
            clear_output(wait=True)
            selected_pair = audio_llm_selector.value
            audio_path_str = audio_file_input.value
            language = audio_language_input.value

            # Vérification simple
            if not os.path.exists(audio_path_str):
                print(f"⚠️ Erreur : Fichier introuvable : {audio_path_str}")
                return

            try: 
                provider, model_id = selected_pair.split(":", 1)
                print(f"🎤 Starting transcription via {provider} with {model_id}...")
                
                result = connector.invoke_audio(
                    provider=provider, model_id=model_id, 
                    audio_file_path=audio_path_str, language=language, debug=False
                )
                
                text = result.get("text", "Pas de texte retourné.")
                # Nettoyage si c'est du JSON brut
                if isinstance(text, str) and text.strip().startswith("{") and "text" in text:
                     text = json.loads(text).get("text", text)

                display(Markdown(f"### ✅ Transcription :\n---\n{text}"))
            except Exception as e:
                print(f"❌ Erreur : {e}")
    
    audio_invoke_button.on_click(on_invoke_audio_clicked)
    
    tab_audio = widgets.VBox([
        audio_llm_selector, audio_file_input, audio_language_input, 
        audio_invoke_button, audio_output_area
    ])

    # =================================================================
    # TAB 9: VOICE CHAT (Full Audio Pipeline)
    # =================================================================

    # --- AJOUT IMPORTANT : Paramètres du micro ---
    SAMPLE_RATE = 44100  # Qualité standard CD
    CHANNELS = 1         # 1 pour Mono (suffisant pour la voix)
    # ---------------------------------------------
    
    voice_chat_llm_selector = widgets.Dropdown(
        options=MODELS_BY_MODALITY["text"], 
        value='openai:gpt-4o', 
        description='LLM Model:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    voice_chat_tts_selector = widgets.Dropdown(
        options=['alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer'], 
        value='nova', description='TTS Voice:', 
        style={'description_width': 'initial'}, 
        layout=Layout(width='50%')
    )
    voice_chat_duration_slider = widgets.IntSlider(
        value=5, min=2, max=10, step=1, 
        description='Duration (sec):', 
        style={'description_width': 'initial'}
    )
    voice_chat_invoke_button = widgets.Button(
        description='🚀 Press and Speak', 
        button_style='success', 
        layout=Layout(margin='10px 0 0 55px', width='80%', height='50px')
    )
    voice_chat_output_area = widgets.Output(layout=Layout(margin='10px'))

    def on_invoke_voice_chat_clicked(b):
        with voice_chat_output_area:
            clear_output(wait=True)
            llm_pair = voice_chat_llm_selector.value
            tts_voice = voice_chat_tts_selector.value
            duration = voice_chat_duration_slider.value
            llm_provider, llm_model_id = llm_pair.split(":", 1)
            audio_in_file = "voice_input_unified.wav" 

            try:
                # Step 1: Record
                print(f"\n📢 Recording STARTED. Speak for {duration} seconds...")
                recording = sd.rec(int(duration * SAMPLE_RATE), 
                                  samplerate=SAMPLE_RATE, channels=CHANNELS, dtype='int16')
                sd.wait() 
                write(audio_in_file, SAMPLE_RATE, recording)  
                print(f"✅ Recording saved to: {audio_in_file}")

                # Step 2: Transcription
                print("\n⏳ Step 2: Transcription (STT)...")
                stt_result = connector.invoke_audio(
                    provider="infomaniak", model_id="whisper", 
                    audio_file_path=audio_in_file, language="en", debug=False
                )
                transcribed_text = stt_result.get("text", "").strip()
                if not transcribed_text:
                    print("🛑 Transcription failed. Stopping.")
                    return
                print(f"📝 Transcribed: \"{transcribed_text}\"")

                # Step 3: LLM
                print(f"\n⏳ Step 3: LLM processing with {llm_model_id}...")
                messages = [{"role": "user", "content": transcribed_text}]
                llm_output = connector.invoke(
                    provider=llm_provider, model_id=llm_model_id, 
                    messages=messages, temperature=0.7, debug=False
                )
                response_text = llm_output.get("text", "Sorry, I couldn't respond.")
                print(f"🤖 LLM Response: \"{response_text}\"")

                # Step 4: TTS
                print("\n⏳ Step 4: Speech synthesis (TTS)...")
                tts_result = connector.invoke_speech_generation(
                    provider="openai", model_id="tts-1", 
                    text=response_text, voice=tts_voice, 
                    response_format="mp3", debug=False
                )
                audio_bytes = tts_result.get("audio_bytes")
                if not audio_bytes:
                    print("❌ Speech synthesis failed.")
                    return

                # Step 5: Save and play
                timestamp = datetime.now().strftime("%Y_%m_%d_%H%M%S")
                safe_model_id = llm_model_id.replace(":", "_")
                filename = f"{timestamp}_voicechat_{safe_model_id}_{tts_voice}.mp3"
                final_output_path = OUTPUT_DIRS["speech"] / filename
                with open(final_output_path, "wb") as f:
                    f.write(audio_bytes)
                print(f"✅ Response audio saved to: {final_output_path}")
                print(f"🎧 Playing response...")

                IN_COLAB = 'google.colab' in sys.modules
                if IN_COLAB:
                    display(Audio(data=audio_bytes))
                else:
                    if pygame.mixer.get_init():
                        try:
                            pygame.mixer.music.load(final_output_path)
                            pygame.mixer.music.play()
                            while pygame.mixer.music.get_busy():
                                time.sleep(0.1)
                            print("✅ Playback completed.")
                        except Exception as e:
                            print(f"❌ Playback error: {e}")
                    else:
                        print("⚠️ pygame.mixer not initialized.")

            except Exception as e:
                display(Markdown(f"### ❌ Voice Chat Error"))
                print(f"Detail: {e}")
    
    voice_chat_invoke_button.on_click(on_invoke_voice_chat_clicked)
    
    # Colab warning
    IN_COLAB = 'google.colab' in sys.modules
    voice_chat_widgets_list = [
        widgets.HTML("<h3>🗣️🎙️ Voice Chat (Full Audio Pipeline)</h3>"
                    "<p>Speak, AI transcribes, processes, and responds vocally.</p>"),
    ]

    if IN_COLAB:
        colab_warning = widgets.HTML(
            value="""<div style='background-color: #fffbe6; border: 1px solid #ffe58f; 
                    padding: 10px; margin: 10px 0; border-radius: 4px;'>
            <b>⚠️ Feature Unavailable on Google Colab</b>
            <p style='margin: 5px 0 0 0;'>
            Audio recording (via <code>sounddevice</code>) cannot access your microphone
            from Colab's server.
            </p>
            <p style='margin: 5px 0 0 0;'>
            <b>To use voice chat:</b> run this notebook locally (e.g., in VS Code).
            </p>
            </div>"""
        )
        voice_chat_widgets_list.append(colab_warning)
        voice_chat_llm_selector.disabled = True
        voice_chat_tts_selector.disabled = True
        voice_chat_duration_slider.disabled = True
        voice_chat_invoke_button.disabled = True
        voice_chat_invoke_button.description = "🚀 (Unavailable on Colab)"
        voice_chat_invoke_button.button_style = ''

    voice_chat_widgets_list.extend([
        voice_chat_llm_selector, voice_chat_tts_selector, 
        voice_chat_duration_slider, voice_chat_invoke_button, voice_chat_output_area
    ])
    tab_voice_chat = widgets.VBox(voice_chat_widgets_list)

    # =================================================================
    # FINAL ASSEMBLY
    # =================================================================

    tab_widget = widgets.Tab(children=[
        tab_text, tab_chat, tab_benchmark, tab_vision, 
        tab_image_gen, tab_video_gen, tab_speech_gen, tab_audio, tab_voice_chat
    ])

    # Set tab titles
    tab_widget.set_title(0, f"✍️ Text ({len(MODELS_BY_MODALITY['text'])} models)")
    tab_widget.set_title(1, f"💬 Chat multi-turn") 
    tab_widget.set_title(2, f"📊 Benchmark") 
    tab_widget.set_title(3, f"👁️ Vision ({len(MODELS_BY_MODALITY['vision'])} models)")
    tab_widget.set_title(4, f"🎨 Image Gen ({len(MODELS_BY_MODALITY['image_gen'])} models)")
    tab_widget.set_title(5, f"🎬 Video Gen ({len(MODELS_BY_MODALITY['video_gen'])} models)")
    tab_widget.set_title(6, f"🗣️ TTS ({len(MODELS_BY_MODALITY['speech_gen'])} models)")
    tab_widget.set_title(7, f"🎤 STT ({len(MODELS_BY_MODALITY['audio'])} models)") 
    tab_widget.set_title(8, f"🗣️🎙️ Voice Chat") 

    # CSS styling
    style = """
    <style>
        .lm-TabBar-tab {
            min-width: 130px !important; 
            max-width: 180px !important; 
            padding-left: 8px !important; 
            padding-right: 8px !important;
            flex: 1 1 auto !important;   
        }
        .lm-TabBar-tabLabel {
            white-space: normal !important; 

        }
        .chat-output p {
            font-size: 1em !important; 
            font-weight: normal !important; 
            margin: 0 !important;
            padding: 0 !important;
        }
        .chat-output b {
            font-weight: bold !important; 
        }
    </style>
    """
    
    # Display interface
    display(HTML(style))
    display(Markdown("# 🤖 Universal Testing Interface"))
    display(Markdown("*Quick-test all modalities with interactive widgets*"))
    display(tab_widget)


## 🎓 Summary

You've learned how to:

✅ Install and import `ideal-ai` as a proper package  
✅ Use text generation with multiple providers  
✅ Analyze images with vision models  
✅ Generate images from text  
✅ Transcribe audio to text  
✅ Synthesize speech from text  
✅ Generate videos (with async polling)  
✅ Add custom models at runtime  
✅ Integrate with smolagents  
✅ Debug API calls  

## 🚀 Next Steps

1. **Extend config.json** - Add your favorite models without Python code
2. **Build agents** - Combine multiple modalities in workflows
3. **Deploy** - Use in production with proper error handling

## 📖 Documentation

- [GitHub Repository](https://github.com/Devgoodcode/ideal-ai)
- [Connector API](https://github.com/Devgoodcode/ideal-ai/blob/main/src/ideal_ai/connector.py) - Full method signatures with docstrings
- [Configuration Schema](https://github.com/Devgoodcode/ideal-ai/blob/main/src/ideal_ai/config.json) - Available families and models
- [Examples](https://github.com/Devgoodcode/ideal-ai/tree/main/examples) - Working code samples

## 📺 See it in action

[![Watch the Demo](https://img.youtube.com/vi/f1DwFRpo2HA/0.jpg)](https://www.youtube.com/watch?v=f1DwFRpo2HA)

> *One Connector to Rule Them All. Watch the full demo (2.50 min).*

## 👤 Author & Support

**Gilles Blanchet**
- 🛠️ Created by: [IA-Agence.ai](https://ia-agence.ai/ideal-ai-universal-llm-connector/) - *Need help integrating Generative AI? Let's talk.*
- 🌐 Agency: [Idealcom.ch](https://idealcom.ch)
- 🐙 GitHub: [@Devgoodcode](https://github.com/Devgoodcode)
- 💼 LinkedIn: [Gilles Blanchet](https://www.linkedin.com/in/gilles-blanchet-566ab759/)

## 🙏 Acknowledgments

This project is a labor of love, built on the shoulders of giants. Special thanks to:

* **🤗 Hugging Face**: For the fantastic *Agents Course*. It inspired me to create this connector to easily apply their concepts using my own existing tools (like Ollama & Infomaniak) without the hassle of writing wrappers.
* **My AI Co-pilots & Mentors**:
    * **Microsoft Copilot**: For the architectural breakthroughs (Families & Invoke concepts) and our late-night debates.
    * **Perplexity**: For laying down the initial code foundation.
    * **Google Gemini**: For the massive refactoring, patience, and pedagogical support in improving the core logic.
    * **Kilo Code (Kimi & Claude)**: For the security testing, English translation, and PyPI publishing preparation.
* **The Model Providers**: Ollama, Alibaba, Moonshot, MiniMax, OpenAI, and Infomaniak for their incredible technologies and platforms.
* **The Open Source Community**: For the endless passion and knowledge sharing.

Built with ❤️ and passion, inspired by the open source AI community's need for a truly universal, maintainable LLM interface.

*The adventure is just beginning...*

---

**One Connector to Rule Them All** 🧙‍♂️